# Importamos nuestras dependencias

In [1]:
# Principales dependencias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Tratamos de entender el problema

### Obtenemos los datso del dataset

In [2]:
# Lo primero es importar el dataset.
# Como ahora tenemos 3 hay que recordar como funciona
# TRAIN viene siendo lo que era el IRIS dataset
# Lo dividimos en train y test
# Dentro de la porcion de test incluimos el dataset TEST que tendrá todas las columnas excepto Survived (Y predecida)
# Realizaremos las predicciones y completamos la columna Survived dentro del archivo TEST
# Esta informacion la utilizamos para finalmente trabajar con el dataset GENDER SBMITION 
# Realizamos la prediccion de los sobrevivientes por sexo.

df = pd.read_csv('datasets/train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### Borro columna PassegerID

In [4]:
# Borro la columna  porque no aporta nada a la prediccion
# Mencionno el nombre de la columna como parámetro y axis=1
df = df.drop('PassengerId', axis = 1)

# Exploratory Data analysis (EDA)

In [5]:
# Esta es una parte muy importante del trabajo, saber que es la data que estaremos manejando
# Es bueno dediccar tiempo a entender si nos faltan muchos datos para decidir que columnas aportan a la prediccion.

In [6]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Featuring Engineering

En esta parte podemos hacer uso de la informacion obtenida y conclusiones

Para hacerlo lo más simple posible, lo que haremos será elegir solamente algunas columnas.

In [7]:
# Aqui cargo las columnas para hacer la parte de predicciones
# Observamos el tipo de informacion que tenemos
# Veremos que la coumna name no sirve para la prediccion y aporta muchos datos
# Entonces si no me sirve la quito 
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
# Observamos los datos que nos faltan
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

Ahora comenzamos a trabajar cada columna

#### 1- Name

In [9]:
# Este es un truco para no tener que eliminar una columna cuando no la queremos utilizar
# Al parecer clasifica con un saludo de acuerdo con un rango de edad ***
df['Name'] = df['Name'].str.extract('([A-Za-z]+)\.',expand=False)

In [10]:
df.head()


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,Mrs,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
# Pero por ahora es mejor eliminar la columna porque a la hora de utilizar los algoritmos 
# NO se puede trabar con ella como un float siendo un string

df = df.drop('Name', axis = 1)

#### 2- Age

Usamos el valor promedio de la collumna para rellenar los valores

In [12]:
df.Age.isnull().sum()

177

In [13]:
# Sustituyo los datos que faltan de la edad por la media
# fillna  ------> rellenar los NA con la media
df.Age = df.Age.fillna(df.Age.mean())

In [14]:
# Como ya lleno los datos con lamedia si lo hago de nuevo no saldra ninguno NA
df.Age.isnull().sum()

0

In [15]:
df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C


#### 3- Ticket

No la tendremos en cuenta por simplificar

In [16]:
df.Ticket.value_counts()

347082      7
CA. 2343    7
1601        7
3101295     6
CA 2144     6
           ..
9234        1
19988       1
2693        1
PC 17612    1
370376      1
Name: Ticket, Length: 681, dtype: int64

In [17]:
# Borro la columna  porque no aporta nada a la prediccion
# Mencionno el nombre de la columna como parámetro y axis=1
df = df.drop('Ticket', axis = 1)

#### 4- Cabin

No la tendremos en cuante porque falta mucha información

In [18]:
# La informacion de Cabin es muy rara y no la utilizamos mejor. Quitamos la columna
df.Cabin.isnull()


0       True
1      False
2       True
3      False
4       True
       ...  
886     True
887    False
888     True
889    False
890     True
Name: Cabin, Length: 891, dtype: bool

In [19]:
# Borro la columna  porque no aporta nada a la prediccion
# Mencionno el nombre de la columna como parámetro y axis=1
df = df.drop('Cabin', axis = 1)

#### 5- Embarked

In [20]:
# Nos fijamos cuantos datos faltan
# Como nos faltan tan pocos los podemos rellenar fácilmente 
df.Embarked.isnull().sum()

2

In [21]:
# Vemos como están colocados los datos y el tipo de dato para ver el que más de repite
# Esto lo hacemos para saber con qué rellenar los datos que faltan
df.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [22]:
# Entonces vamos a rellenar los datos que faltan con una S, porque como se ve arriba es la que más se repite del total de los demás
df['Embarked']=df['Embarked'].fillna('S')

In [23]:
df.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [24]:
df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


 Las columnas de SibSp y Parch no las vamos a utilizar

In [25]:
# Una vez que tenemos nuestro DataFrame(df)
# Lo siguiente será trabajar con las columnas que tienen texto

Concepto de  **datos categoricos:**

- columnas con strings hombre/mujer por ejemplo
- columnas con strings con 3 opciones por ejemplo
- en el caso de Pclas 3 hace referencia a "tercera clase"
- y 3 no vale más que 1, ymás en este case, cuya porbabilidad de supervivencia es más baja

In [26]:
# pd.get_dummies?

In [27]:
# Tenemos que hacer el cambio de categorico a numerico por ejemplo en la columna de Sex
# De hecho si en sex decimos que es Hombre por defecto ya no es mujer
# Esto es importante para acabar analizando una sola columna que nos permite entender la informacion de varias
# Estras tres columnas las trabajamos para que aparesca con 0 y 1 BOOLEANO
# Estas columnas tienen que ser booleanas para poder analizarlas 

# Al final quito la primera columan para que no hayan problemas con Multicolinealidad
df = pd.get_dummies(df,columns=['Sex','Pclass','Embarked'], drop_first=True)
df.head()

,Survived,Age,SibSp,Parch,Fare,Sex_male,Pclass_2,Pclass_3,Embarked_Q,Embarked_S
0,0,22.0,1,0,7.2500,1,0,1,0,1
1,1,38.0,1,0,71.2833,0,0,0,0,0
2,1,26.0,0,0,7.9250,0,0,1,0,1
3,1,35.0,1,0,53.1000,0,0,0,0,1
4,0,35.0,0,0,8.0500,1,0,1,0,1


In [28]:
# Aqui lo que podemos observar es que unas coumans tienes un rango (Age) y otras tienen otro rango (fare)
# Entonces es necesario escalarlas

### Escalado de datos

Existen varias formas de hacer el escalado de datos . Normallmente no hay diferencias, pero algunas veces si.

Por abreviar trataremos de mencionar 2 tipo( en Sklearn)

- StandardScaler
- MinMaxScaler

En nuestro caso no daremos importancia a cuál es mejor

Tampoco usaremos la propia libreria, sino la propia ecuación

Existen varias formas de hacerlo, pero lo haremos de forma simple, y antes de dividir el train/test

In [29]:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [30]:
# StandardScaler


# Esta es la formula matematica
#  X menos la media
# X-mean(x)/std(x)   ---------------------------> X = X.values/ny =y.values

df.Age = (df.Age - np.mean(df.Age, axis=0))/ (np.std(df.Age, axis=0))
df.Fare = (df.Fare - np.mean(df.Fare, axis=0))/ (np.std(df.Fare, axis=0))
df.head()

,Survived,Age,SibSp,Parch,Fare,Sex_male,Pclass_2,Pclass_3,Embarked_Q,Embarked_S
0,0,-0.592481,1,0,-0.502445,1,0,1,0,1
1,1,0.638789,1,0,0.786845,0,0,0,0,0
2,1,-0.284663,0,0,-0.488854,0,0,1,0,1
3,1,0.407926,1,0,0.420730,0,0,0,0,1
4,0,0.407926,0,0,-0.486337,1,0,1,0,1


### Obtenemos X y Y

In [31]:
# Es el caso de el Iris data set igual obteniamos X ([pl,pw,sl,sw]) y Y(especies)
# En este caso es igual pero cambia X y Y

# X ---------> Es todo menos la salida que es Y 

X = df.drop('Survived', axis=1)
X.head()


,Age,SibSp,Parch,Fare,Sex_male,Pclass_2,Pclass_3,Embarked_Q,Embarked_S
0,-0.592481,1,0,-0.502445,1,0,1,0,1
1,0.638789,1,0,0.786845,0,0,0,0,0
2,-0.284663,0,0,-0.488854,0,0,1,0,1
3,0.407926,1,0,0.420730,0,0,0,0,1
4,0.407926,0,0,-0.486337,1,0,1,0,1


In [32]:
y = df['Survived']
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

### Entrenamiento y prueba

In [33]:
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

# Aqui tenemos 4 matrices 
# Estas son los cortes que hemos realizado en el eje X
# Las dos de arriba de entrenamiento
# Las dos de abajo de prueba

# Los parametros son:
# X ------> INPUT
# y ------> OUTPUT

# Test size significa que es unarelacion de 20-80
# random state es para hacerlo seudoaleatorio con una replicabilidad
# Asi el resultado se puede replicar cuantas veces queramos.
# Esto significa que los revuelve para no cojer solo los datso de arriba para entrenamiento
# Aveces esto podria no entrenar bien el modelo

X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
X_train.head()

,Age,SibSp,Parch,Fare,Sex_male,Pclass_2,Pclass_3,Embarked_Q,Embarked_S
331,1.215947,0,0,-0.074583,1,0,0,0,1
733,-0.515526,0,0,-0.386671,1,1,0,0,1
382,0.177063,0,0,-0.488854,1,0,1,0,1
704,-0.284663,1,0,-0.490280,1,0,1,0,1
813,-1.823750,4,2,-0.018709,0,0,1,0,1


In [35]:
y_train.head()

331    0
733    0
382    0
704    0
813    0
Name: Survived, dtype: int64

In [36]:
y_test.head()

709    1
439    0
840    0
720    1
39     1
Name: Survived, dtype: int64

### Pruebo posibles algoritmos

In [37]:
# KNeighborClassifier

clf = KNeighborsClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
acc_KN = accuracy_score(y_test, y_pred)
acc_KN

0.8212290502793296

In [38]:
# DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
acc_DT = accuracy_score(y_test, y_pred)
acc_DT

0.776536312849162

In [39]:
# RandomForestClasifier
clf = RandomForestClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
acc_RF = accuracy_score(y_test, y_pred)
acc_RF

0.8212290502793296

In [40]:
# GaussianNB
clf = GaussianNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
acc_NB = accuracy_score(y_test, y_pred)
acc_NB

0.7653631284916201

In [41]:
# SVC
clf = GaussianNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
acc_SVC = accuracy_score(y_test, y_pred)
acc_SVC

0.7653631284916201

### Busco el que mejor a priori mejor predice

In [42]:
# KNeighborClassifier

# Una vexz que decida cual es el algoritmo que mejor se adapta. Entreno con este algoritmo 
#Le tengo que pasar TEST
# Pero ahora en lugar de pasarle solo la parte de entrenamiento le paso TODO TEST.CSV

clf = KNeighborsClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
acc_KN = accuracy_score(y_test, y_pred)
acc_KN

0.8212290502793296

## Uso este entrenamiento para el archivo test.csv

In [43]:
# Pero para ello primero necesito realizar una serie de pasos:
# Leer el archivo
# Hacer los mismos cambios que hice en train
# Cuando hago los cambios ya tendré el mismo número de columnas en uno que el otro
test = pd.read_csv('datasets/test.csv')
test.head(2)

# Como podemos ver aqui no tengo la columna de survived

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [44]:
# En este dataset original tengo la columna de SURVIDED pero en el ede test ya no lo tenia
df_original = pd.read_csv('datasets/train.csv')
df_original.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


## Llama la atención algo ..

### Ahora ya no tengo la información de SURVIVED

### Tendremos que predecirla

#### Pasos:

- HAremos a cada columna exactamente los mismos cambios en "Test" que en df"
- Haremos la predicción (ahora no sabemos de momento como de buena o mala fue)
- Nos iremos a Kaggle para enviar los resultados



In [45]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


## Age

Usamos el valor promedio de la collumna para rellenar los valores

In [46]:
# Sustituyo los datos que faltan de la edad por la media
# fillna  ------> rellenar los NA con la media
test.Age = test.Age.fillna(test.Age.mean())

In [47]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

##  Fare

In [48]:
# Vemos como están colocados los datos y el tipo de dato para ver el que más de repite
# Esto lo hacemos para saber con qué rellenar los datos que faltan
test.Fare.value_counts()

7.7500     21
26.0000    19
13.0000    17
8.0500     17
7.8958     11
           ..
7.8208      1
8.5167      1
78.8500     1
52.0000     1
22.3583     1
Name: Fare, Length: 169, dtype: int64

In [49]:
# Entonces vamos a rellenar los datos que faltan con una , porque como se ve arriba es la que más se repite del total de los demás
test['Fare']=test['Fare'].fillna(-0.498101)

In [50]:
# Ahora elimino las columnas que no voy a utilizar

test = test.drop(['Name','Ticket','Cabin'], axis=1)
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


### Escalamos los datos como lo hicimos con el primer dataset

In [51]:
# df.Age = (df.Age - np.mean(df.Age, axis=0))/ (np.std(df.Age, axis=0))
#df.Fare = (df.Fare - np.mean(df.Fare, axis=0))/ (np.std(df.Fare, axis=0))

test.Age = (test.Age - np.mean(test.Age, axis=0))/ (np.std(test.Age, axis=0))
test.Fare = (test.Fare - np.mean(test.Fare, axis=0))/ (np.std(test.Fare, axis=0))
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,0.334993,0,0,-0.496609,Q
1,893,3,female,1.325530,1,0,-0.511469,S
2,894,2,male,2.514175,0,0,-0.463307,Q
3,895,3,male,-0.259330,0,0,-0.481676,S
4,896,3,female,-0.655545,1,1,-0.416713,S


In [52]:
# Tenemos que hacer el cambio de categorico a numerico por ejemplo en la columna de Sex
# De hecho si en sex decimos que es Hombre por defecto ya no es mujer
# Esto es importante para acabar analizando una sola columna que nos permite entender la informacion de varias
# Estras tres columnas las trabajamos para que aparesca con 0 y 1 BOOLEANO
# Estas columnas tienen que ser booleanas para poder analizarlas 

# Al final quito la primera columan para que no hayan problemas con Multicolinealidad
# df = pd.get_dummies(df,columns=['Sex','Pclass','Embarked'], drop_first=True)

test = pd.get_dummies(test, columns=['Sex','Pclass','Embarked'], drop_first= True)

In [53]:
# df.Age = df.Age.fillna(df.Age.mean())

#test.Age = test.Age.fillna(test.Age.mean())
#test.Fare = test.Fare.fillna(test.Fare.mean())

In [54]:
test.head()
test.isnull().sum()

PassengerId    0
Age            0
SibSp          0
Parch          0
Fare           0
Sex_male       0
Pclass_2       0
Pclass_3       0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [55]:
# Tambien quitamos la columna de Passenger Id
test = test.drop('PassengerId', axis=1)
test.head()


,Age,SibSp,Parch,Fare,Sex_male,Pclass_2,Pclass_3,Embarked_Q,Embarked_S
0,0.334993,0,0,-0.496609,1,0,1,1,0
1,1.325530,1,0,-0.511469,0,0,1,0,1
2,2.514175,0,0,-0.463307,1,1,0,1,0
3,-0.259330,0,0,-0.481676,1,0,1,0,1
4,-0.655545,1,1,-0.416713,0,0,1,0,1


Ya lo tenemos todo listo para aplicar el algoritmo

In [56]:
# KNeighborClassifier fue el que se adaptó mejor a los datos
clf = KNeighborsClassifier()

# Entreno con los datos que tenía ne el PRIMER DATASET
clf.fit(X_train,y_train)
#######################
# Ahora hago la predicción sobre test
# Durante el entrenamiento lo haciamos sobre X_test ---------- y_pred = clf.predict(X_test)
# En lugar de pasarle X test le pasamos todo la matriz de test.
y_predecida = clf.predict(test)
y_predecida

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [57]:
# Este es el resultado de Y_PREDECIDA
# Este resultado lo tengo que colocar en el archivo de GENDER_SUBMISSION

# Si nos fijamos bien los que nos arroja es un array de Numpy
type(y_predecida)

numpy.ndarray

### Me creo un dataframe con esa información

In [58]:
df_submission = pd.read_csv('datasets/gender_submission.csv')
# Este dataframe viene con unos valores por defecto en la columna de SURVIVED
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [59]:
# Le asigno a la columna Survived los valores de y_predecida
# Este resultado lo obtuve alimentando el algoritmo con los valores de test
df_submission['Survived'] = y_predecida
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


## Paso a .CSV

In [60]:
# IMPORTANTE de colocar el index FALSE para cuando queremos subir la preduiccion a Kaggle que no nos estorbe
# Evitamos problemas y se hace rápido
# Paso al archivo CSV
df_submission.to_csv('datasets/titanic_solucion.csv', index= False)